<a href="https://colab.research.google.com/github/batel-arnon/python_from_school/blob/main/Face_Recognition_pt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **imports**

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Model
!pip install tflocalpattern
from tflocalpattern.layers import LDP
!pip install tkinter

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **the full code:**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import to_categorical

# Load the LFW dataset
lfw_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4) # maybe shold try: (40,1.0)
#print(lfw_dataset)
# Extract features (images) and labels from the dataset
X = lfw_dataset.images
y = lfw_dataset.target

# Reshape input data to include the channel dimension
X = X.reshape(-1, X.shape[1], X.shape[2], 1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # maybe should try test_size = 0.33

# Preprocess the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define your face recognition model architecture
def create_face_recognition_model(input_shape, num_classes):
    # Define the input layer with the specified input shape
    inputs = layers.Input(shape=input_shape)
    print(input_shape)
    # Add convolutional layers
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten the feature maps
    x = layers.Flatten()(x)

    # Add a fully connected layer with dropout for regularization
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Output layer with softmax activation for classification
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Define the model
    model = Model(inputs, outputs)

    return model

# Create an instance of the face recognition model
input_shape = X_train[0].shape
model = create_face_recognition_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) # maybe should try loss =  'sparse_categorical_crossentropy'

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test)) # maybe should try it until the epochs only ( without ths batch_size and validation_data)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


In [ ]:
model.save('drive/MyDrive/modelgpt')

In [ ]:
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

def preprocess_image(image, target_size=(50, 37)):

    # Convert the image to grayscale
    image = image.convert('L')  # 'L' mode stands for grayscale

    # Resize the image
    image = image.resize(target_size[::-1], resample=Image.ANTIALIAS)

    # Convert the resized image to numpy array
    image_array = np.array(image)

    # Normalize pixel values
    image_array = image_array / 255.0

    # Expand dimensions to match model input shape
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.expand_dims(image_array, axis=-1)  # Add channel dimension for grayscale images

    return image_array

In [ ]:
def index_of_the_max_value(arr):
  index = 0
  for i in range(len(arr)):
    if arr[i]>arr[index]:
      index = i
  return index

def recognized_name(image):
  model = load_model('/content/drive/MyDrive/modelgpt')
  #image = Image.open(image_path)
  #image.show()
  target_size = (50, 37)
  image = preprocess_image(image, target_size)
  print(image)
  recognized_faces = model.predict(image)
  print(recognized_faces)
  index = index_of_the_max_value(recognized_faces[0])
  target_names = ['Ariel Sharon', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush',
       'Gerhard Schroeder', 'Hugo Chavez', 'Tony Blair']
  return target_names[index]

image = Image.open('/content/hugo chavez.jpeg')
recognized_name(image)

# **design**

In [ ]:
# Gradio UI
!pip install gradio
import gradio as gr

interface = gr.Interface(fn=recognized_name, inputs="image", outputs="label")
interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8639c4022a46388da5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1788, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

code to compile the model:

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(125,94)),  # input layer (1)
    keras.layers.Dense(128, activation='relu'),  # hidden layer (2)

    keras.layers.Dense(10, activation='softmax') # output layer (3)
])